In [2]:
import numpy as np

Utilizaremos los siguientes generadores de números uniformes:
- `XORShift (64 bits)`
- `Mersenne Twister (MT19937)`
- `Permuted congruential generator (64-bit, PCG64 DXSM)` 

In [ ]:
class Xorshift64:
    def __init__(self, seed):
        if seed == 0:
            raise ValueError("Seed must be non-zero")
        self.state = seed

    def next(self):
        x = self.state
        x ^= (x << 13) & 0xFFFFFFFFFFFFFFFF
        x ^= (x >> 7) & 0xFFFFFFFFFFFFFFFF
        x ^= (x << 17) & 0xFFFFFFFFFFFFFFFF
        self.state = x & 0xFFFFFFFFFFFFFFFF
        return self.state

In [ ]:
def MT19937(seed):
    """
    Mersenne Twister (MT19937) to generate a pseudo-random variable.  
    Returns a Generator object.
    """
    return np.random.Generator(bit_generator=np.random.MT19937(seed))

In [ ]:
def PCG64(seed):
    """
    Permuted Congruential Generator (PCG64) to generate a pseudo-random variable.  
    Returns a Generator object.
    """
    return np.random.Generator(bit_generator=np.random.PCG64(seed))

In [ ]:
import time

class RNG:
    """
    ## Random Number Generator (RNG)  
    Use this class to generate a random number using some of the available generators:
    - xorshift64
    - mt19937
    - pcg64
    
    ### Usage:
    Without passing seed:  
    r = **RNG**("xorshift64")  
    r.random()
    
    Passing seed:  
    r = **RNG**("xorshift64", seed=1234)  
    r.random()
    """
    def __init__(self, algo='xorshift64', seed=None):
        if seed is None:
            seed = int(time.time_ns())
        self.algo = algo.lower()

        if self.algo == 'xorshift64':
            if seed == 0:
                raise ValueError("Seed must be non-zero for Xorshift64")
            self.generator = Xorshift64(seed)
        elif self.algo == 'mt19937':
            self.generator = MT19937(seed)
        elif self.algo == 'pcg64':
            self.generator = PCG64(seed)
        else:
            raise ValueError(f"Unknown algorithm: {self.algo}")

    def random(self):
        if self.algo == 'xorshift64':
            return self.generator.next() / 2**64
        else:
            return self.generator.random()

#### Intensidad del proceso Poisson homogéneo
$$
    \lambda(t) = 30 + 30 \cdot \sin\left(\frac{2\pi t}{24}\right) \text{ (clientes/hora)}
$$

In [ ]:
# Intensidad del Poisson proceso homogéneo
def lambda_t(t):
    """
    Intensity function for a homogeneous Poisson process.
    """
    return 30 + 30 * np.sin((2*np.pi*t)/24)


#### Tiempo de Atención

In [ ]:
def service_time(gen: RNG):
    return -np.log(gen.random()) / 40

#### Servidor

In [ ]:
from queuelib.queue import FifoMemoryQueue as Queue

def server(t_arrivals: list, gen: RNG):
    t_server_available = 0
    wait_queue = Queue()
    
    # process all events
    for t_curr, t_next in zip(t_arrivals, t_arrivals[1:]):
        t_arrival = t_curr
        
        if t_arrival >= t_server_available:
            # server is available
            t_start = t_arrival
            t_service = service_time(gen)
            t_end = t_start + t_service
            
            t_server_available = t_end

            # process events on queue
            while len(wait_queue) and t_server_available <= t_next:
                wait_queue.pop()
                t_start = t_server_available
                t_service = service_time(gen)
                t_end = t_start + t_service
            
                t_server_available = t_end
                
        else:
            # server is unavailable
            wait_queue.push(t_curr)

#### Generación de eventos

In [ ]:
def events_generator(gen: RNG) -> list:
    t = 0
    arrivals = []
    T_max = 48
    lambda_max = 60
    
    while True:
        # generate an interarrival time candidate
        u1 = gen.random()
        t += -np.log(1 - u1) / lambda_max
        
        # terminate if generated time is bigger than T_max bound
        if t > T_max:
            break
        
        # decide if t is accepted based on lambda(t) / lambda_max relation
        u2 = gen.random()
        if u2 < lambda_t(t) / lambda_max:
            arrivals.append(t)
    
    return arrivals